In [ ]:
#default_exp utils.misc

In [ ]:
#export

import os, sys
import inspect
import logging
import json

# Debug setup

In [ ]:
#DEBUG = os.environ.get('GITHUB_ACTIONS', False) or os.environ.get('AUDIOENGINE_DEBUG', False)
DEBUG = True

# Object attribute util functions

In [ ]:
#export

def recursive_get_size(obj, seen=None):
    size = sys.getsizeof(obj)
    seen = (set() if seen is None else seen)
    if(id(obj) in seen): return 0 #In the case of a self reference
    else: pass
    seen.add(id(obj))
    if(isinstance(obj, dict)):
        size += sum([recursive_get_size(v, seen=seen) for v in obj.values()])
        size += sum([recursive_get_size(k ,seen=seen) for k in obj.keys()])
    elif(hasattr(obj, '__dict__')):
        size += recursive_get_size(obj.__dict__, seen)
    elif(hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray))):
        size += sum([recursive_get_size(i, seen) for i in obj])
    return size

# Logging utils

In [ ]:
#export

def log_init(log_filepath: str='', log_level: str='', clear_existing_log: bool=False):
    config_filepath = os.environ.get('AUDIOENGINE_CONF_JSON', '/project/Development/ML/audio/config/audioengine.conf')
    if(os.path.isfile(config_filepath)):
        config_fp = open(config_filepath, 'r')
        config_str = config_fp.read()
        config_fp.close()
    else:
        config_str = ''
    config_json = json.loads(config_str)
    logging_options_json = config_json.get('logging', None)
    if(logging_options_json is not None and (log_filepath == '' or log_level == '')):
        if(log_level == ''):
            log_level = logging_options_json.get('log_level')
        else:
            pass
        if(log_filepath == ''):
            log_filepath = logging_options_json.get('log_file')
        else:
            pass
    else:
        if(log_level == ''):
            log_level = 'warning'
        else:
            pass
        #Don't do anything for the filepath because it needs to log to console
        
    filemode = ('w' if clear_existing_log else 'a')
    
    if(log_level.lower() == 'debug'):
        if(log_filepath == ''):
            logging.basicConfig(level=logging.DEBUG, filemode=filemode)
        else:
            logging.basicConfig(level=logging.DEBUG, filename=log_filepath, filemode=filemode)
    elif(log_level.lower() == 'info'):
        if(log_filepath == ''):
            logging.basicConfig(level=logging.INFO, filemode=filemode)
        else:
            logging.basicConfig(level=logging.INFO, filename=log_filepath, filemode=filemode)
    elif(log_level.lower() == 'warning'):
        if(log_filepath == ''):
            logging.basicConfig(level=logging.WARNING, filemode=filemode)
        else:
            logging.basicConfig(level=logging.WARNING, filename=log_filepath, filemode=filemode)
    elif(log_level.lower() == 'error'):
        if(log_filepath == ''):
            logging.basicConfig(level=logging.ERROR, filemode=filemode)
        else:
            logging.basicConfig(level=logging.ERROR, filename=log_filepath, filemode=filemode)
    elif(log_level.lower() == 'critical'):
        if(log_filepath == ''):
            logging.basicConfig(level=logging.CRITICAL, filemode=filemode)
        else:
            logging.basicConfig(level=logging.CRITICAL, filename=log_filepath, filemode=filemode)
    else:
        #The log_level wasn't matched
        logging.basicConfig(level=logging.WARNING)
    log_location = ('console' if log_filepath == '' else 'file: {}'.format(log_filepath))
    print('[INFO] Logging now set to {} with level {}'.format(log_location, log_level.upper()))
        

def log_critical(message, exc_info=False):
    "Automatically log the current function details."
    func = inspect.currentframe().f_back.f_code
    # Dump the message + the name of this function to the log.
    logging.critical("{}: {} in {}:{}".format(func.co_filename, func.co_name, func.co_firstlineno, message),
                     exc_info=exc_info)

def log_error(message, exc_info=False):
    "Automatically log the current function details."
    func = inspect.currentframe().f_back.f_code
    # Dump the message + the name of this function to the log.
    logging.error("{}: {} in {}:{}".format(func.co_filename, func.co_name, func.co_firstlineno, message),
                  exc_info=exc_info)
    
def log_warning(message):
    "Automatically log the current function details."
    func = inspect.currentframe().f_back.f_code
    # Dump the message + the name of this function to the log.
    logging.warning("{}: {} in {}:{}".format(func.co_filename, func.co_name, func.co_firstlineno, message))    
    
def log_info(message):
    "Automatically log the current function details."
    func = inspect.currentframe().f_back.f_code
    # Dump the message + the name of this function to the log.
    logging.info("{}: {} in {}:{}".format(func.co_filename, func.co_name, func.co_firstlineno, message))
    
def log_debug(message):
    "Automatically log the current function details."
    func = inspect.currentframe().f_back.f_code
    # Dump the message + the name of this function to the log.
    logging.debug("{}: {} in {}:{}".format(func.co_filename, func.co_name, func.co_firstlineno, message))

In [ ]:
if DEBUG:
    #log_init()
    log_init(clear_existing_log=True)
    log_critical('Critical test')
    log_error('Error test')
    log_warning('Warning test')
    log_info('Info test')
    log_debug('Debug test')
    
    try:
        a = 2 / 0
    except Exception as exception:
        log_critical('Exception raised', exc_info=True)
        
    try:
        a = 2 / 0
    except Exception as exception:
        log_error('Exception raised', exc_info=True)

[INFO] Logging now set to file: /project/Development/ML/audio/logs/audioengine.log with level DEBUG
